In [9]:
import pandas as pd
import numpy as np

# Load the dataset from the provided CSV file
df = pd.read_csv('faa_ai_prelim.csv')

# Display the first 5 rows to understand the data
print("Initial Data Preview:")
print(df.head())

# Get a summary of the dataset, including data types and null values
print("\nDataset Information:")
df.info()

Initial Data Preview:
  UPDATED ENTRY_DATE EVENT_LCL_DATE EVENT_LCL_TIME LOC_CITY_NAME  \
0      No  19-FEB-16      19-FEB-16      00:45:00Z    MARSHVILLE   
1      No  19-FEB-16      18-FEB-16      23:55:00Z     TAVERNIER   
2      No  19-FEB-16      18-FEB-16      22:14:00Z       TRENTON   
3      No  19-FEB-16      18-FEB-16      17:10:00Z     ASHEVILLE   
4      No  19-FEB-16      18-FEB-16      00:26:00Z     TALKEETNA   

   LOC_STATE_NAME LOC_CNTRY_NAME  \
0  North Carolina            NaN   
1         Florida            NaN   
2      New Jersey            NaN   
3  North Carolina            NaN   
4          Alaska            NaN   

                                            RMK_TEXT EVENT_TYPE_DESC  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...        Accident   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...        Incident   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...        Incident   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...        Inci

In [10]:
# Combine date and time columns into a single string
# The .str.replace('Z', '', regex=False) removes the 'Z' suffix from the time string
datetime_str = df['EVENT_LCL_DATE'] + ' ' + df['EVENT_LCL_TIME'].str.replace('Z', '', regex=False)

# Convert the combined string to datetime objects
# errors='coerce' will place NaT (Not a Time) for any rows that fail to parse
df['Event_Datetime'] = pd.to_datetime(datetime_str, errors='coerce')

# Verify the new column and its data type
print("\nDataFrame with new 'Event_Datetime' column:")
print(df[['EVENT_LCL_DATE', 'EVENT_LCL_TIME', 'Event_Datetime']].head())
print("\nData type of the new column:")
print(df['Event_Datetime'].dtype)


DataFrame with new 'Event_Datetime' column:
  EVENT_LCL_DATE EVENT_LCL_TIME      Event_Datetime
0      19-FEB-16      00:45:00Z 2016-02-19 00:45:00
1      18-FEB-16      23:55:00Z 2016-02-18 23:55:00
2      18-FEB-16      22:14:00Z 2016-02-18 22:14:00
3      18-FEB-16      17:10:00Z 2016-02-18 17:10:00
4      18-FEB-16      00:26:00Z 2016-02-18 00:26:00

Data type of the new column:
datetime64[ns]


C:\Users\athar\AppData\Local\Temp\ipykernel_3700\1959727372.py:7: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Event_Datetime'] = pd.to_datetime(datetime_str, errors='coerce')


In [16]:
# Define the list of columns required by the assignment
required_columns = [
    'ACFT_MAKE_NAME',
    'LOC_STATE_NAME',
    'ACFT_MODEL_NAME',
    'RMK_TEXT',
    'FLT_PHASE',
    'EVENT_TYPE_DESC',
    'FATAL_FLAG',
    'ACFT_DMG_DESC' # Added based on later steps in the assignment
]

# Create the new dataframe with only the specified columns
processed_df = df[required_columns].copy()

print("\nNew DataFrame with required columns:")
print(processed_df.head())


New DataFrame with required columns:
  ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0          BEECH  North Carolina              36   
1           VANS         Florida             RV7   
2         CESSNA      New Jersey             172   
3        LANCAIR  North Carolina             235   
4         CESSNA          Alaska             172   

                                            RMK_TEXT       FLT_PHASE  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   UNKNOWN (UNK)   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   LANDING (LDG)   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...  APPROACH (APR)   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...   LANDING (LDG)   
4  AIRCRAFT ON LANDING, NOSE GEAR COLLAPSED, TALK...   LANDING (LDG)   

  EVENT_TYPE_DESC FATAL_FLAG ACFT_DMG_DESC  
0        Accident        Yes       Unknown  
1        Incident        NaN       Unknown  
2        Incident        NaN         Minor  
3        Incident        NaN       U

In [17]:
print("\nMissing values before handling:")
print(processed_df.isnull().sum())

# 1. Replace all Fatal Flag missing values with 'No'
processed_df['FATAL_FLAG'].fillna('No', inplace=True)

# 2. For the FLT_PHASE and ACFT_DMG_DESC columns, use the mode to fill missing data
flt_phase_mode = processed_df['FLT_PHASE'].mode()[0]
processed_df['FLT_PHASE'].fillna(flt_phase_mode, inplace=True)

acft_dmg_mode = processed_df['ACFT_DMG_DESC'].mode()[0]
processed_df['ACFT_DMG_DESC'].fillna(acft_dmg_mode, inplace=True)

print("\nMissing values after handling:")
print(processed_df.isnull().sum())


Missing values before handling:
ACFT_MAKE_NAME      5
LOC_STATE_NAME      0
ACFT_MODEL_NAME     4
RMK_TEXT            0
FLT_PHASE           1
EVENT_TYPE_DESC     0
FATAL_FLAG         75
ACFT_DMG_DESC       9
dtype: int64

Missing values after handling:
ACFT_MAKE_NAME     5
LOC_STATE_NAME     0
ACFT_MODEL_NAME    4
RMK_TEXT           0
FLT_PHASE          0
EVENT_TYPE_DESC    0
FATAL_FLAG         0
ACFT_DMG_DESC      0
dtype: int64


C:\Users\athar\AppData\Local\Temp\ipykernel_3700\4125945434.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  processed_df['FATAL_FLAG'].fillna('No', inplace=True)
C:\Users\athar\AppData\Local\Temp\ipykernel_3700\4125945434.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For 

In [ ]:
print(f"\nNumber of observations before dropping data: {len(processed_df)}")

# Remove all the observations where aircraft names are not available
initial_rows = len(processed_df)
processed_df.dropna(subset=['ACFT_MAKE_NAME'], inplace=True)
rows_after_dropna = len(processed_df)
print(f"Dropped {initial_rows - rows_after_dropna} rows with missing aircraft names.")


# Drop columns with a high percentage of missing values (interpreted from the assignment)
# We will drop columns that are more than 75% null.
print("\nChecking for sparse columns in the original DataFrame:")
null_percentage = df.isnull().sum() / len(df) * 100
columns_to_drop = null_percentage[null_percentage > 75].keys()
print(f"Columns with >75% null values to be dropped: {list(columns_to_drop)}")
# df.drop(columns=columns_to_drop, inplace=True) # This would be the code to drop them


print(f"\nNumber of observations now: {len(processed_df)}")



### **Step 6: Data Aggregation and Filtering**


# Group the dataset by aircraft name and view the number of times each appears
aircraft_counts = processed_df.groupby('ACFT_MAKE_NAME').size().sort_values(ascending=False)
print("\nDataset grouped by aircraft name (count of incidents):")
print(aircraft_counts)

# Display the observations where the fatal flag is “Yes”
fatal_incidents = processed_df[processed_df['FATAL_FLAG'] == 'Yes']
print("\nObservations where Fatal Flag is 'Yes':")
print(fatal_incidents)


Number of observations before dropping data: 83
Dropped 5 rows with missing aircraft names.

Checking for sparse columns in the original DataFrame:
Columns with >75% null values to be dropped: ['LOC_CNTRY_NAME', 'FLT_NBR', 'ACFT_OPRTR', 'ACFT_MISSING_FLAG', 'FLT_ACTIVITY', 'FAR_PART', 'MAX_INJ_LVL', 'FATAL_FLAG', 'FLT_CRW_INJ_MINOR', 'FLT_CRW_INJ_SERIOUS', 'FLT_CRW_INJ_FATAL', 'FLT_CRW_INJ_UNK', 'CBN_CRW_INJ_NONE', 'CBN_CRW_INJ_MINOR', 'CBN_CRW_INJ_SERIOUS', 'CBN_CRW_INJ_FATAL', 'CBN_CRW_INJ_UNK', 'PAX_INJ_NONE', 'PAX_INJ_MINOR', 'PAX_INJ_SERIOUS', 'PAX_INJ_FATAL', 'PAX_INJ_UNK', 'GRND_INJ_NONE', 'GRND_INJ_MINOR', 'GRND_INJ_SERIOUS', 'GRND_INJ_FATAL', 'GRND_INJ_UNK']

Number of observations now: 78

Dataset grouped by aircraft name (count of incidents):
ACFT_MAKE_NAME
CESSNA                    23
PIPER                     10
BEECH                      9
MOONEY                     4
BOEING                     3
CHAMPION                   2
SOCATA                     2
LANCAIR          

In [21]:
print("\nDataFrame columns before One-Hot Encoding:")
print(processed_df.columns)
print("\nUnique values in 'ACFT_DMG_DESC':", processed_df['ACFT_DMG_DESC'].unique())


# Use one-hot encoding on the ACFT_DMG_DESC column
processed_df_encoded = pd.get_dummies(processed_df, columns=['ACFT_DMG_DESC'], drop_first=True)

print("\nDataFrame after One-Hot Encoding:")
print(processed_df_encoded.head())

print("\nNew columns created by One-Hot Encoding:")
print(processed_df_encoded.columns)


DataFrame columns before One-Hot Encoding:
Index(['ACFT_MAKE_NAME', 'LOC_STATE_NAME', 'ACFT_MODEL_NAME', 'RMK_TEXT',
       'FLT_PHASE', 'EVENT_TYPE_DESC', 'FATAL_FLAG', 'ACFT_DMG_DESC'],
      dtype='object')

Unique values in 'ACFT_DMG_DESC': ['Unknown' 'Minor' 'Substantial' 'Destroyed']

DataFrame after One-Hot Encoding:
  ACFT_MAKE_NAME  LOC_STATE_NAME ACFT_MODEL_NAME  \
0          BEECH  North Carolina              36   
1           VANS         Florida             RV7   
2         CESSNA      New Jersey             172   
3        LANCAIR  North Carolina             235   
4         CESSNA          Alaska             172   

                                            RMK_TEXT       FLT_PHASE  \
0  AIRCRAFT CRASHED INTO TREES, THE 1 PERSON ON B...   UNKNOWN (UNK)   
1  AIRCRAFT ON LANDING WENT OFF THE END OF THE RU...   LANDING (LDG)   
2  AIRCRAFT ON FINAL SUSTAINED A BIRD STRIKE, LAN...  APPROACH (APR)   
3  AIRCRAFT ON LANDING, GEAR COLLAPSED, ASHEVILLE...   LANDING (LDG)   


In [22]:
# Advanced Task: Feature Engineering from Unstructured Text

# Define the keywords to search for
phase_keywords = ["LANDING", "TAKEOFF", "APPROACH", "CRUISE", "TAXI", "STANDING"]

# Create a function to search for these keywords in the text
def extract_phase(text):
    # Ensure text is a string
    if not isinstance(text, str):
        return 'UNKNOWN'
    
    text_upper = text.upper()
    for keyword in phase_keywords:
        if keyword in text_upper:
            return keyword
    return 'UNKNOWN'

# Apply the function to the RMK_TEXT column to create the new feature
processed_df['FLIGHT_PHASE_TEXT'] = processed_df['RMK_TEXT'].apply(extract_phase)

# Compare the newly created feature with the existing FLT_PHASE column
# We will clean up the original FLT_PHASE column for a better comparison
processed_df['FLT_PHASE_CLEAN'] = processed_df['FLT_PHASE'].str.extract(r'([A-Z]+)')[0]

comparison = pd.crosstab(processed_df['FLT_PHASE_CLEAN'], processed_df['FLIGHT_PHASE_TEXT'])

print("\nComparison of original FLT_PHASE with new feature extracted from RMK_TEXT:")
print(comparison)


Comparison of original FLT_PHASE with new feature extracted from RMK_TEXT:
FLIGHT_PHASE_TEXT  APPROACH  LANDING  TAKEOFF  TAXI  UNKNOWN
FLT_PHASE_CLEAN                                             
APPROACH                  1        0        0     0        1
LANDING                   0       32        0     0       12
STANDING                  0        0        0     0        3
TAKEOFF                   0        0        4     0        2
TAXI                      0        1        0     9        0
UNKNOWN                   0        0        0     0       13
